In [1]:
import pyNN.neuron as sim
import matplotlib
matplotlib.use("Agg")
from neuroml import Morphology, Segment, Point3DWithDiam as P
from pyNN.morphology import NeuroMLMorphology, uniform
from pyNN.parameters import IonicSpecies
#from pyNN.units import uF_per_cm2, ohm_cm, S_per_cm2, mV, nA, ms
from pyNN.utility import get_simulator
from pyNN.utility.plotting import Figure, Panel

In [2]:
sim.setup(timestep=0.025)

0

In [3]:
soma = Segment(proximal=P(x=0, y=0, z=0, diameter=18.8),
               distal=P(x=18.8, y=0, z=0, diameter=18.8),
               name="soma", id=0)
dend = Segment(proximal=P(x=0, y=0, z=0, diameter=2),
               distal=P(x=-500, y=0, z=0, diameter=2),
               name="dendrite",
               parent=soma, id=1)

In [4]:
cell_class = sim.MultiCompartmentNeuron
cell_class.label = "ExampleMultiCompartmentNeuron"
cell_class.ion_channels = {'pas': sim.PassiveLeak, 'na': sim.NaChannel, 'kdr': sim.KdrChannel}

In [5]:
cell_type = cell_class(morphology=NeuroMLMorphology(Morphology(segments=(soma, dend))),  # yuck
                       cm=1.0,
                       Ra=500.0,
                       ionic_species={
                              "na": IonicSpecies("na", reversal_potential=50.0),
                              "k": IonicSpecies("k", reversal_potential=-77.0)
                       },
                       pas={"conductance_density": uniform('all', 0.0003),
                            "e_rev":-54.3},
                       na={"conductance_density": uniform('soma', 0.120),
                           "e_rev": 50.0},
                       kdr={"conductance_density": uniform('soma', 0.036),
                            "e_rev": -77.0}
                       )

In [6]:
cell_type.model

pyNN.neuron.standardmodels.cells.ExampleMultiCompartmentNeuron

In [11]:
cell_type.model.post_synaptic_entities

{}

In [16]:
cell_type.model.ion_channels

{'pas': PassiveLeak(<parameters>),
 'na': NaChannel(<parameters>),
 'kdr': KdrChannel(<parameters>)}

In [12]:
#cells = sim.Population(2, cell_type, initial_values={'v': [-60.0, -70.0]})
class X:
    a = 1

In [13]:
X = type('X', (), dict(a=1))

In [15]:
X.a

1

Therefore,
```
    def model(self):
        return type(self.label,
                    (NeuronTemplate,),
                    {"ion_channels": self.ion_channels,
                     "post_synaptic_entities": self.post_synaptic_entities})
```
returns the template given by the label name having the two attributes

The cell building does not take place either in MultiCompartmentNeuron or NeuronTemplate (although this class has the receipe to build). It takes place in Population where the method `_create_cells` invokes `_build_cell` for every cell in the population; this method takes the first argument as the NeuronTemplate class and the second argument is the same argument as you would if you were to invoke NeuronTemplate in isolation. The `_build_cell` method is defined in the `ID` class in `simulatior.py` file. Here, the second argument is passed as the argument to the NeuronTemplate class (the first argument) resulting in creating a cell object assigned to the attribute `self._cell`.